# Сравнение оптимизаторов нейросетки для модели wordchar2vector с помощью ANOVA

#### Наша задача - определить лучший оптимизатор из [доступных на Keras](https://keras.io/optimizers/) и подтвердить выбор статистическими тестами. Модель - нейросетка, которая учится генерировать векторы для символьного представления слов (см. [описание](https://github.com/Koziev/chatbot/blob/master/PyModels/trainers/README.wordchar2vector.md)). Особенность нейросетевых моделей заключается в существенной стохастичности решения из-за нюансов начальной инициализации сетки, действий оптимизатора и случайного перемешивания данных в датасетах. Поэтому сравнение средней точности моделей для разных оптимизаторов мы дополним статистическими тестами. 

In [8]:
from __future__ import print_function

import random
import codecs
import itertools
import os
import numpy as np
import pandas as pd
import math
from scipy import stats
import matplotlib.pyplot as plt
from utils.tokenizer import Tokenizer

%matplotlib inline

### Списки файлов с историей обучения, сохраненных тренером модели

In [14]:
# файлы лежат в ../tmp

# оптимизатор nadam
nadam_files = ['learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=123456.csv',
               'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=654321.csv',
               'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=654321_1.csv',
               'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=456789.csv',
               'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=5678901.csv']

# оптимизатор rmsprop
rmsprop_files = ['learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=10000234.csv',
                 'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=10013987.csv',
                 'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=10003987.csv',
                 'learning_curve__lstm(cnn)_vecsize=56_tunable_char_embeddings=0_chardims=87_batchsize=250_seed=10001987.csv',
                ]

In [15]:
def find_max_accuracy(dataframe_filename):
    """Получим максимальное значение достигнутой instance accuracy в каждом эксперименте"""
    df = pd.read_csv(os.path.join(u'../tmp', dataframe_filename), header=None, encoding='utf-8', delimiter='\t', index_col=None)
    max_acc = np.amax(df[1].values)
    return max_acc

def get_samples(dataframe_filenames):
    return [find_max_accuracy(fname) for fname in dataframe_filenames]


In [16]:
# Получим выборки отдельно для каждого оптимизатора
samples_nadam = get_samples(nadam_files)
samples_rmsprop = get_samples(rmsprop_files)

In [17]:
print(samples_nadam)
print(samples_rmsprop)

[0.92000000000000004, 0.91900000000000004, 0.91799999999999993, 0.93000000000000005, 0.94299999999999995]
[0.83900000000000008, 0.873, 0.80400000000000005, 0.88700000000000001]


### ANOVA: односторонний F-тест для проверки нулевой гипотезы

#### Проверяем гипотезу: выборочные средние равны, а отличия вызваны случайными причинами (высокой стохастичностью процесса обучения).

In [18]:
import scipy.stats

res = scipy.stats.f_oneway(samples_nadam, samples_rmsprop)
print('one-way p={}'.format(res.pvalue))

one-way p=0.00322564771963


#### При доверительном интервале 0.05 получаемое значение p говорит о том, что риск ошибки при отказе от нулевой гипотезы достаточно мал